In [ ]:
!git clone https://github.com/Mateusz-Wojciechowski/sentimentAnalysis.git
%cd sentimentAnalysis

fatal: destination path 'sentimentAnalysis' already exists and is not an empty directory.
/content/sentimentAnalysis


In [ ]:
!pip install 'portalocker>=2.0.0'

In [ ]:
import torch
from torchtext.datasets import IMDB
from torchtext.data.utils import get_tokenizer
from torchtext.vocab import build_vocab_from_iterator
from torch.utils.data import DataLoader
from torch.nn.utils.rnn import pad_sequence
from functools import partial
from SentimentModel import SentimentModel
import torch.optim as optim
import torch.nn as nn

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)
# Tokenizer
tokenizer = get_tokenizer('basic_english')

def yield_tokens(data_iter):
    for text in data_iter:
        yield tokenizer(text)

def process_text(text, vocab):
    return torch.tensor(vocab(tokenizer(text)), dtype=torch.long)

def collate_batch(batch):
    label_list, text_list = [], []
    for label, text in batch:
        label_tensor = torch.tensor([label-1], dtype=torch.float)
        processed_text = process_text(text, vocab)
        label_list.append(label_tensor)
        text_list.append(processed_text)
    return torch.stack(label_list), pad_sequence(text_list, padding_value=vocab["<pad>"], batch_first=True)

def calculate_accuracy(preds, y):
    preds = torch.sigmoid(preds)
    rounded_preds = torch.round(preds)
    correct = (rounded_preds == y).float()
    accuracy = correct.sum() / len(correct)
    return accuracy


train_data = list(IMDB(split='train'))
random.shuffle(train_data)

vocab = build_vocab_from_iterator(yield_tokens(data_iter for label, data_iter in train_data), specials=["<unk>", "<pad>"])
vocab.set_default_index(vocab["<unk>"])

train_loader = DataLoader(train_data, batch_size=32, shuffle=False, collate_fn=collate_batch)

test_data = list(IMDB(split='test'))
random.shuffle(test_data)
test_loader = DataLoader(test_data, batch_size=32, shuffle=False, collate_fn=collate_batch)

d_model = 512
num_heads = 8
max_seq_len = 5000
d_ff = 2048
learning_rate = 0.001
num_classes = 2
vocab_size = len(vocab)
num_epochs = 100


model = SentimentModel(d_model, d_ff, num_heads, max_seq_len, num_classes, vocab_size)
optimizer = optim.Adam(model.parameters(), lr=learning_rate)
loss_fn = nn.BCEWithLogitsLoss()


for epoch in range(num_epochs):
    print(f"Epoch: {epoch + 1}")
    total_loss = 0
    total_accuracy = 0
    total_examples = 0

    for labels, sequences in train_loader:
        labels, sequences = labels.to(device), sequences.to(device)

        output = model(sequences)
        loss = loss_fn(output, labels)
        accuracy = calculate_accuracy(output, labels)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        total_loss += loss.item()
        total_accuracy += accuracy.item()
        total_examples += labels.size(0)

    print(f"Loss in epoch {epoch + 1} is {total_loss}")
    print(f"Accuracy in epoch {epoch + 1} is {total_accuracy / total_examples}")

Using device: cuda
batch 0
batch 10
batch 20
batch 30
batch 40
batch 50
batch 60
batch 70
batch 80
batch 90
batch 100
batch 110
batch 120
batch 130
batch 140
batch 150
batch 160
batch 170
batch 180
batch 190
batch 200
batch 210
batch 220
batch 230
batch 240
batch 250
batch 260
batch 270
batch 280
batch 290
batch 300
batch 310
batch 320
batch 330
batch 340
Loss in epoch 0 is 0.4778189799710191
batch 0
batch 10
batch 20
batch 30
batch 40
batch 50
batch 60
batch 70
batch 80
batch 90
batch 100
batch 110
batch 120
batch 130
batch 140
batch 150
batch 160
batch 170
batch 180
batch 190
batch 200
batch 210
batch 220
batch 230
batch 240
batch 250
batch 260
batch 270
batch 280
batch 290
batch 300
batch 310
batch 320
batch 330
batch 340
Loss in epoch 1 is 4.462897280177458e-05
batch 0
batch 10
batch 20
batch 30
batch 40
batch 50
batch 60
batch 70
batch 80
batch 90
batch 100
batch 110
batch 120
batch 130
batch 140
batch 150
batch 160
batch 170
batch 180
batch 190
batch 200
batch 210
batch 220
batch

KeyboardInterrupt: ignored